# Gaussian Iterative Stockholder Analysis (GISA) method

In [ ]:
import logging

from scipy.optimize import least_squares
from setup import *

from horton_part import GaussianISAWPart

logging.basicConfig(level=logging.INFO, format="%(levelname)s:    %(message)s")

## Quadratic Programming

### Using quadprog solver

The problem of this solver is that the initials values are set to zeros for all pro-atom parameters and no threshold setting is availiable.

In [ ]:
mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")


def qp_method():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = 1
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)


qp_method()

### Using SLSQP sovler

A equivalent method is using `minimization` with the 'SLSQP' sovler implemented in `SciPy` package. 

In [ ]:
mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")


def qp_method_scipy():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = 2
    kwargs[
        "inner_threshold"
    ] = 1e-10  # the value is from the test file of quadprog package.
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)


qp_method_scipy()

### Using CVXOPT solver
Another sovler implemented in `CVXOPT` can also be applied.

In [ ]:
mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")


def qp_method_cvxopt():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = 3
    kwargs["inner_threshold"] = 1e-8
    part = GaussianISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    opt_propars = part._cache["propars"]
    print("Difference between total electrons:")
    print(grid.integrate(rho) - np.sum(opt_propars))


qp_method_cvxopt()

## Customized Least-Square Sovler

Users can easily apply customized solvers. The first step involves defining a customized solver, for example, `customized_solver`. The corresponding arguments and their meanings are as follows:

In [ ]:
def customized_solver(bs_funcs, rho, propars, points, weights, alphas, threshold):
    r"""
    Optimize pro-atom parameters using quadratic-programming implemented in the `CVXOPT` package.

    .. math::

        N_{Ai} = \int \rho_A(r) \frac{\rho_{Ai}^0(r)}{\rho_A^0(r)} dr

        G = \frac{1}{2} c^T S c - c^T b

        S = 2 \int \zeta(\vec{r}) \zeta(\vec{r}) d\vec{r}
        = \frac{2}{\pi \sqrt{\pi}} \frac{(\alpha_k \alpha_l)^{3/2}}{(\alpha_k + \alpha_l)^{3/2}}

        b = \int \zeta(\vec{r}) \rho_a(\vec{r}) d\vec{r}

    Parameters
    ----------
    bs_funcs : 2D np.ndarray
        Basis functions array with shape (M, N), where 'M' is the number of basis functions
        and 'N' is the number of grid points.
    rho : 1D np.ndarray
        Spherically-averaged atomic density as a function of radial distance, with shape (N,).
    propars : 1D np.ndarray
        Pro-atom parameters with shape (M). 'M' is the number of basis functions.
    points : 1D np.ndarray
        Radial coordinates of grid points, with shape (N,).
    weights : 1D np.ndarray
        Weights for integration, including the angular part (4πr²), with shape (N,).
    threshold : float
        Convergence threshold for the iterative process.
    alphas : 1D np.ndarray
        The Gaussian exponential coefficients.
    threshold : float
        The convergence threshold of the optimization method.

    Returns
    -------
    1D np.ndarray
        Optimized proatom parameters.

    Raises
    ------
    RuntimeError
        If the inner iteration does not converge.

    """

    def _obj_func(x):
        pro = np.sum(x[:, None] * bs_funcs, axis=0)
        return np.sqrt(weights) * np.abs(pro - rho)

    result = least_squares(
        _obj_func, x0=np.zeros_like(propars), bounds=(0, np.inf), ftol=threshold
    )
    return result.x

Next, we need to pass this customized solver to our partitioning method.

In [ ]:
def customized_method():
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = customized_solver
    kwargs["inner_threshold"] = 1e-14
    part = GaussianISAWPart(**kwargs)
    part.do_all()

    print_results(part)
    opt_propars = part._cache["propars"]
    print("Difference between total electrons:")
    print(grid.integrate(rho) - np.sum(opt_propars))


customized_method()

The result of the least squares method differs slightly from the quadratic programming method due to the constraints. We can observe that the deviation in the total electrons obtained by the least squares method is larger than that obtained by the quadratic programming method.

## General Quadratic Programming Interface: qpsovlers

`qpsolvers` is a general interafce to call different quadratic programming solvers, e.g., `quadprog` or `cvxopt`.

In [ ]:
import qpsolvers


def opt_propars_qp_qpsolvers(
    bs_funcs,
    rho,
    propars,
    points,
    weights,
    alphas,
    threshold,
    solver="cvxopt",
    **kwargs,
):
    """
    Optimize pro-atom parameters using quadratic-programming implemented in the `CVXOPT` package.

    Parameters
    ----------
    bs_funcs : 2D np.ndarray
        Basis functions array with shape (M, N), where 'M' is the number of basis functions
        and 'N' is the number of grid points.
    rho : 1D np.ndarray
        Spherically-averaged atomic density as a function of radial distance, with shape (N,).
    propars : 1D np.ndarray
        Pro-atom parameters with shape (M). 'M' is the number of basis functions.
    points : 1D np.ndarray
        Radial coordinates of grid points, with shape (N,).
    weights : 1D np.ndarray
        Weights for integration, including the angular part (4πr²), with shape (N,).
    threshold : float
        Convergence threshold for the iterative process.
    alphas : 1D np.ndarray
        The Gaussian exponential coefficients.
    threshold : float
        The convergence threshold of the optimization method.
    solver : str
        The name of sovler. See `qpsovler.solve_qp`.

    Returns
    -------
    1D np.ndarray
        Optimized proatom parameters.

    Raises
    ------
    RuntimeError
        If the inner iteration does not converge.

    """
    nprim, npt = bs_funcs.shape

    P = (
        2
        / np.pi**1.5
        * (alphas[:, None] * alphas[None, :]) ** 1.5
        / (alphas[:, None] + alphas[None, :]) ** 1.5
    )
    P = (P + P.T) / 2

    q = -2 * np.einsum("i,ni,i->n", weights, bs_funcs, rho)

    # Linear inequality constraints
    G = -np.identity(nprim)
    h = np.zeros((nprim, 1))

    # Linear equality constraints
    A = np.ones((1, nprim))
    pop = np.einsum("i,i", weights, rho)
    b = np.ones((1, 1)) * pop

    # initial_values = cvxopt.matrix(np.array([1.0] * nprim).reshape((nprim, 1)))
    result = qpsolvers.solve_qp(
        P,
        q,
        G,
        h,
        A,
        b,
        solver=solver,
        initvals=np.zeros_like(propars),
        # threshold=threshold,
        # eps_rel=threshold,
        # eps_abs=threshold * 10,
        # eps_prim_inf=threshold,
        **kwargs,
    )
    return result

In [ ]:
def qpsolvers_interface(solver, **kwargs):
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = opt_propars_qp_qpsolvers
    kwargs["solver_kwargs"] = {"solver": solver}
    part = GaussianISAWPart(**kwargs)
    part.do_all()

    print_results(part)
    opt_propars = part._cache["propars"]
    print(opt_propars)
    print("Difference between total electrons:")
    print(grid.integrate(rho) - np.sum(opt_propars))


# 'cvxopt', 'daqp', 'ecos', 'osqp', 'quadprog', 'scs'

In [ ]:
qpsolvers_interface("quadprog")

In [ ]:
qpsolvers_interface("cvxopt", eps_rel=1e-8)

In [ ]:
qpsolvers_interface("ecos", reltol=1e-8)

In [ ]:
qpsolvers_interface("osqp", eps_rel=1e-8)